<a href="https://colab.research.google.com/github/Ashvin7/pl-xg-ml/blob/main/01_phase1_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 1: Data Ingestion + First Clean Dataset (2017–18 to 2024–25)

In Phase 1, I will:
- Load manually-downloaded FBref CSVs for each season (League Table + Squad Standard Stats)
- Standardize column names and team naming
- Validate row counts and missingness
- Merge into one team-season dataset
- Export a single clean CSV into `data/processed/` for Phase 2


In [13]:
import os, re, glob, shutil
import pandas as pd

os.makedirs("/content/data/raw", exist_ok=True)
os.makedirs("/content/data/processed", exist_ok=True)

print("Folders ready:",
      os.path.exists("/content/data/raw"),
      os.path.exists("/content/data/processed"))

Folders ready: True True


## Fix: Ensure CSVs are in the right folder

Colab sometimes puts uploaded files in `/content/sample_data/`.
This cell will:
1) find all `epl_*.csv` anywhere under `/content/`
2) move them into `/content/data/raw/`
3) confirm we have the expected counts


In [30]:
raw_dir = "/content/data/raw"

# Find any EPL csvs anywhere under /content
found = glob.glob("/content/**/epl_*.csv", recursive=True)

print("Found EPL CSVs anywhere under /content:", len(found))
print("Sample:", found[:5])

# Move them into data/raw (avoid duplicating by overwriting or skipping)
moved = 0
for src in found:
    dst = os.path.join(raw_dir, os.path.basename(src))
    if os.path.abspath(src) == os.path.abspath(dst):
        continue
    # If dst exists, overwrite (safe because these are source CSVs)
    if os.path.exists(dst):
        os.remove(dst)
    shutil.move(src, dst)
    moved += 1

print("Moved into data/raw:", moved)

# Confirm
raw_now = sorted(glob.glob(f"{raw_dir}/epl_*.csv"))
print("Now in data/raw:", len(raw_now))
print("Sample:", raw_now[:5])

Found EPL CSVs anywhere under /content: 16
Sample: ['/content/data/raw/epl_2021-22_squad_standard.csv', '/content/data/raw/epl_2024-25_squad_standard.csv', '/content/data/raw/epl_2022-23_squad_standard.csv', '/content/data/raw/epl_2021-22_league_table.csv', '/content/data/raw/epl_2022-23_league_table.csv']
Moved into data/raw: 0
Now in data/raw: 16
Sample: ['/content/data/raw/epl_2017-18_league_table.csv', '/content/data/raw/epl_2017-18_squad_standard.csv', '/content/data/raw/epl_2018-19_league_table.csv', '/content/data/raw/epl_2018-19_squad_standard.csv', '/content/data/raw/epl_2019-20_league_table.csv']


## Expected input files

For each season (2017–18 through 2024–25), I should have:
- `epl_<season>_league_table.csv`
- `epl_<season>_squad_standard.csv`

That means:
- 8 league table CSVs
- 8 squad standard CSVs


In [15]:
raw_dir = "/content/data/raw"

league_files = sorted(glob.glob(f"{raw_dir}/epl_*_league_table.csv"))
squad_files  = sorted(glob.glob(f"{raw_dir}/epl_*_squad_standard.csv"))

print("League tables:", len(league_files))
print("Squad standard:", len(squad_files))

print("\nLeague files:")
for f in league_files: print(" -", os.path.basename(f))

print("\nSquad files:")
for f in squad_files: print(" -", os.path.basename(f))

if len(league_files) == 0 or len(squad_files) == 0:
    raise ValueError("No input CSVs found in /content/data/raw. Re-run the move/fix cell and confirm your filenames.")


League tables: 8
Squad standard: 8

League files:
 - epl_2017-18_league_table.csv
 - epl_2018-19_league_table.csv
 - epl_2019-20_league_table.csv
 - epl_2020-21_league_table.csv
 - epl_2021-22_league_table.csv
 - epl_2022-23_league_table.csv
 - epl_2023-24_league_table.csv
 - epl_2024-25_league_table.csv

Squad files:
 - epl_2017-18_squad_standard.csv
 - epl_2018-19_squad_standard.csv
 - epl_2019-20_squad_standard.csv
 - epl_2020-21_squad_standard.csv
 - epl_2021-22_squad_standard.csv
 - epl_2022-23_squad_standard.csv
 - epl_2023-24_squad_standard.csv
 - epl_2024-25_squad_standard.csv


In [31]:
def season_from_filename(path: str) -> str:
    """
    Extracts '2017-18' from 'epl_2017-18_league_table.csv'
    """
    base = os.path.basename(path)
    m = re.search(r"epl_(\d{4}-\d{2})_", base)
    if not m:
        raise ValueError(f"Could not parse season from filename: {base}")
    return m.group(1)

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip().lower() for c in df.columns]
    df.columns = [re.sub(r"[\s%()]", "_", c) for c in df.columns]
    df.columns = [re.sub(r"_+", "_", c) for c in df.columns]
    df.columns = [c.strip('_') for c in df.columns]
    return df

def normalize_team_name(s: str) -> str:
    # Keep it simple and consistent; you can expand later if needed.
    return str(s).strip()

def find_first_col(df, candidates):
    cols = set(df.columns)
    for c in candidates:
        if c in cols:
            return c
    return None

def find_header_row(filepath: str, keywords: list, max_rows_to_check: int = 5) -> int:
    """
    Finds the 0-indexed row number that contains one of the keywords (case-insensitive).
    Returns -1 if not found within max_rows_to_check.
    """
    with open(filepath, 'r') as f:
        for i, line in enumerate(f):
            if any(keyword.lower() in line.lower() for keyword in keywords):
                return i
            if i >= max_rows_to_check - 1:
                break
    return -1

## Load: League Table (Target Variable)

From the league table CSVs, I will keep:
- `team`
- `pts` (target variable)
- `xg`, `xga` (league-level)
- `xgd` (optional but useful)

If any season file is missing one of these, I will error out immediately.


In [32]:
league_rows = []

for fn in league_files:
    season = season_from_filename(fn)

    # Robustly find header row for league tables
    header_idx = find_header_row(fn, ["Squad", "Team", "Pts", "xG"])
    if header_idx == -1:
        raise ValueError(f"Could not find a suitable header row in {os.path.basename(fn)} for season {season}.")

    df = pd.read_csv(fn, header=header_idx)
    df = clean_cols(df)

    team_col = find_first_col(df, ["squad", "team"])
    pts_col  = find_first_col(df, ["pts", "points"])
    xg_col   = find_first_col(df, ["xg"])
    xga_col  = find_first_col(df, ["xga"])
    xgd_col  = find_first_col(df, ["xgd"])

    missing = [name for name, col in {
        "team": team_col, "pts": pts_col, "xg": xg_col, "xga": xga_col, "xgd": xgd_col
    }.items() if col is None]

    if missing:
        raise ValueError(f"[League {season}] Missing columns: {missing}. Available: {list(df.columns)}. Header detected at row: {header_idx}")

    out = df[[team_col, pts_col, xg_col, xga_col, xgd_col]].copy()
    out.columns = ["team", "points", "xg_league", "xga_league", "xgd_league"]
    out["team"] = out["team"].map(normalize_team_name)
    out["season"] = season

    # sanity
    if len(out) < 18: # EPL has 20 teams, but promotion/relegation means less data sometimes
        raise ValueError(f"[League {season}] Too few rows ({len(out)}). Did you export the right table?")

    league_rows.append(out)

league_df = pd.concat(league_rows, ignore_index=True)
league_df.head(), league_df.shape

(              team  points  xg_league  xga_league  xgd_league   season
 0  Manchester City     100       78.6        23.8        54.8  2017-18
 1   Manchester Utd      81       55.7        40.7        15.0  2017-18
 2        Tottenham      77       64.7        33.9        30.8  2017-18
 3        Liverpool      75       72.9        33.8        39.1  2017-18
 4          Chelsea      70       54.4        33.8        20.6  2017-18,
 (160, 6))

## Load: Squad Standard Stats (Support Predictors)

From the squad standard CSVs, I will keep:
- `team`
- `mp` (matches played) to normalize xG per match if needed
- `xg`, `xga` from squad standard (these should align closely with league-level, but they’re useful for cross-checking)

I’ll store these as:
- `matches`
- `xg_squad`
- `xga_squad`


In [25]:
squad_rows = []

for fn in squad_files:
    season = season_from_filename(fn)

    # Robustly find header row for squad standard stats
    header_idx = find_header_row(fn, ["Squad", "MP", "xG"])
    if header_idx == -1:
        raise ValueError(f"Could not find a suitable header row in {os.path.basename(fn)} for season {season}.")

    df = pd.read_csv(fn, header=header_idx)
    df = clean_cols(df)

    team_col = find_first_col(df, ["squad", "team"])
    mp_col   = find_first_col(df, ["mp", "matches_played"])
    xg_col   = find_first_col(df, ["xg"])

    missing = [name for name, col in {
        "team": team_col, "mp": mp_col, "xg": xg_col
    }.items() if col is None]

    if missing:
        raise ValueError(f"[Squad {season}] Missing columns: {missing}. Available: {list(df.columns)}. Header detected at row: {header_idx}")

    out = df[[team_col, mp_col, xg_col]].copy()
    out.columns = ["team", "matches", "xg_squad"]
    out["team"] = out["team"].map(normalize_team_name)
    out["season"] = season

    if len(out) < 18:
        raise ValueError(f"[Squad {season}] Too few rows ({len(out)}). Did you export the right table?")

    squad_rows.append(out)

squad_df = pd.concat(squad_rows, ignore_index=True)
squad_df.head(), squad_df.shape

(          team  matches  xg_squad   season
 0      Arsenal       38      68.3  2017-18
 1  Bournemouth       38      38.8  2017-18
 2     Brighton       38      37.0  2017-18
 3      Burnley       38      32.3  2017-18
 4      Chelsea       38      54.4  2017-18,
 (160, 4))

## Merge + Derived Features

I will merge on (`season`, `team`) and compute:
- `xg_diff_league = xg_league - xga_league`
- `xg_diff_squad  = xg_squad - xga_squad`
- `xg_per_match   = xg_squad / matches`
- `xga_per_match  = xga_squad / matches`

Then I will validate expected row counts (8 seasons × 20 teams = 160 rows).


In [26]:
df = league_df.merge(
    squad_df,
    on=["season", "team"],
    how="inner",
    validate="one_to_one"
)

df["xg_diff_league"] = df["xg_league"] - df["xga_league"]
df["xg_per_match"]   = df["xg_squad"]  / df["matches"]

print("Merged shape:", df.shape)
print("Seasons:", sorted(df["season"].unique()))
print("Teams per season (min/max):",
      df.groupby("season")["team"].nunique().min(),
      df.groupby("season")["team"].nunique().max())

# Basic expectation: 8 seasons * 20 teams = 160 rows
# (If a season is missing a team due to naming mismatch, you'll see it here.)
df.head()

Merged shape: (160, 10)
Seasons: ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24', '2024-25']
Teams per season (min/max): 20 20


,team,points,xg_league,xga_league,xgd_league,season,matches,xg_squad,xg_diff_league,xg_per_match
0,Manchester City,100,78.6,23.8,54.8,2017-18,38,78.6,54.8,2.068421
1,Manchester Utd,81,55.7,40.7,15.0,2017-18,38,55.7,15.0,1.465789
2,Tottenham,77,64.7,33.9,30.8,2017-18,38,64.7,30.8,1.702632
3,Liverpool,75,72.9,33.8,39.1,2017-18,38,72.9,39.1,1.918421
4,Chelsea,70,54.4,33.8,20.6,2017-18,38,54.4,20.6,1.431579


from matplotlib import pyplot as plt
_df_0['points'].plot(kind='hist', bins=20, title='points')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['xg_league'].plot(kind='hist', bins=20, title='xg_league')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['xga_league'].plot(kind='hist', bins=20, title='xga_league')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['team'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='xg_league', y='team', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['team'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='xga_league', y='team', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['team'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='xgd_league', y='team', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## Save Phase 1 Dataset

I will write the merged dataset to:

`/content/data/processed/phase1_team_season_dataset.csv`

This is the input to Phase 2 (modeling).


In [29]:
out_path = "/content/data/processed/phase1_team_season_dataset.csv"
df.to_csv(out_path, index=False)
print("Saved:", out_path)

Saved: /content/data/processed/phase1_team_season_dataset.csv
